In [1]:
import json
import numpy as np
import pandas as pd
import psycopg2
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 1000)
pd.set_option("display.float_format", lambda x: "%.2f" % x) # Suppress scientific notation for float data type

## Which drugs are states buying most frequently?

First, we need to determine which drugs appear to be outliers as measured by the number of drugs purchased in each state. To get started, query the Medicaid API to return the drug name, the state that bought the drugs and the total number of drugs purchased in 2016.

In [2]:
query = "https://data.medicaid.gov/resource/neai-csgh.json?$select=state_code,product_fda_list_name,sum(units_reimbursed)&$where=suppression_used=False%20and%20not%20state_code='XX'&$group=state_code,product_fda_list_name&$limit=4625479&$$app_token=v3AK8nRjxbWjtmIBGHJ9OmMlb"
units = pd.read_json(query)
units.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95185 entries, 0 to 95184
Data columns (total 3 columns):
product_fda_list_name    95183 non-null object
state_code               95185 non-null object
sum_units_reimbursed     95185 non-null float64
dtypes: float64(1), object(2)
memory usage: 2.9+ MB


In [3]:
units.head()

,product_fda_list_name,state_code,sum_units_reimbursed
0,ZINC OXIDE,KY,417530.90
1,RAVICTI,TN,15775.00
2,BICILLIN L,IN,1895.51
3,Tramadol H,WA,37041.00
4,NAPROXEN 3,NV,8678.00


Some of our drug names are fully capitalized. Others are not. Since we'll eventually group on that column, we need to standardize that.

In [4]:
units["product_fda_list_name"] = units["product_fda_list_name"].str.upper()
units.head()

,product_fda_list_name,state_code,sum_units_reimbursed
0,ZINC OXIDE,KY,417530.90
1,RAVICTI,TN,15775.00
2,BICILLIN L,IN,1895.51
3,TRAMADOL H,WA,37041.00
4,NAPROXEN 3,NV,8678.00


Rank the drugs by their units reimbursed within each state.

In [5]:
units["rank"] = units.groupby("state_code")["sum_units_reimbursed"].rank(method="min", ascending=False).astype(int)
units.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95185 entries, 0 to 95184
Data columns (total 4 columns):
product_fda_list_name    95183 non-null object
state_code               95185 non-null object
sum_units_reimbursed     95185 non-null float64
rank                     95185 non-null int64
dtypes: float64(1), int64(1), object(2)
memory usage: 6.1+ MB


Create a new dataframe with the top 10 drugs in each state.

In [6]:
top_10_units = units[units["rank"] <= 10]
top_10_units.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 510 entries, 73 to 95170
Data columns (total 4 columns):
product_fda_list_name    510 non-null object
state_code               510 non-null object
sum_units_reimbursed     510 non-null float64
rank                     510 non-null int64
dtypes: float64(1), int64(1), object(2)
memory usage: 19.9+ KB


How many times does each drug appear in a state's top-10 list?

In [7]:
counts_units = top_10_units["product_fda_list_name"].value_counts().reset_index() # Create new dataframe of drug counts
counts_units.columns = ["product_fda_list_name", "count"] # Rename columns
counts_units.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 2 columns):
product_fda_list_name    62 non-null object
count                    62 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.0+ KB


Merge the dataframes into a single dataframe with both ranks and counts.

In [8]:
top_10_units = top_10_units.merge(counts_units, how="inner", on="product_fda_list_name")
top_10_units.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 510 entries, 0 to 509
Data columns (total 5 columns):
product_fda_list_name    510 non-null object
state_code               510 non-null object
sum_units_reimbursed     510 non-null float64
rank                     510 non-null int64
count                    510 non-null int64
dtypes: float64(1), int64(2), object(2)
memory usage: 23.9+ KB


Which drugs appear in only a single state's top-10 list?

In [9]:
outliers_units = top_10_units[top_10_units["count"] == 1]
outliers_units.sort_values("product_fda_list_name", ascending=True)

,product_fda_list_name,state_code,sum_units_reimbursed,rank,count
224,ADVATE 5ML,NV,8649028.00,8,1
500,ALPRAZOLAM,MO,9809834.31,10,1
430,AMLODIPINE,DC,2355515.13,10,1
485,AMMONIUM L,NY,61596452.85,8,1
444,BROMFED DM,TX,62004362.99,7,1
493,BUPROPION,VT,1363162.50,9,1
497,CHILDREN I,TX,77692355.83,6,1
349,CLONAZEPAM,RI,2107665.00,9,1
501,DEXTROAMP-,MA,14595305.50,9,1
486,DEXTROSE 5,WV,12566525.00,8,1


Export the outliers data as an Excel file.

In [10]:
outliers_units.to_excel("data/outliers_units.xlsx")

## Which drugs are states spending the most money on?

So far, we've determined which drugs appear to be outliers as measured by the number of drugs purchased in each state. We now need to determine which drugs appear to be outliers as measured by the total amount of money spent on the drugs. To get started, query the Medicaid API to return the drug name, the state that bought the drugs and the total amount reimbursed for drugs purchased in 2016.

In [11]:
query = "https://data.medicaid.gov/resource/neai-csgh.json?$select=state_code,product_fda_list_name,sum(total_amount_reimbursed)&$where=suppression_used=False%20and%20not%20state_code='XX'&$group=state_code,product_fda_list_name&$limit=4625479&$$app_token=v3AK8nRjxbWjtmIBGHJ9OmMlb"
amount = pd.read_json(query)
amount.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95185 entries, 0 to 95184
Data columns (total 3 columns):
product_fda_list_name          95183 non-null object
state_code                     95185 non-null object
sum_total_amount_reimbursed    95185 non-null float64
dtypes: float64(1), object(2)
memory usage: 2.9+ MB


In [12]:
amount.head()

,product_fda_list_name,state_code,sum_total_amount_reimbursed
0,ZINC OXIDE,KY,44152.93
1,RAVICTI,TN,2396199.88
2,BICILLIN L,IN,48881.32
3,Tramadol H,WA,15972.25
4,NAPROXEN 3,NV,1353.14


Some of our drug names are fully capitalized. Others are not. Since we'll eventually group on that column, we need to standardize that.

In [13]:
amount["product_fda_list_name"] = amount["product_fda_list_name"].str.upper()
amount.head()

,product_fda_list_name,state_code,sum_total_amount_reimbursed
0,ZINC OXIDE,KY,44152.93
1,RAVICTI,TN,2396199.88
2,BICILLIN L,IN,48881.32
3,TRAMADOL H,WA,15972.25
4,NAPROXEN 3,NV,1353.14


Rank the drugs by their amount reimbursed within each state.

In [14]:
amount["rank"] = amount.groupby("state_code")["sum_total_amount_reimbursed"].rank(method="min", ascending=False).astype(int)
amount.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95185 entries, 0 to 95184
Data columns (total 4 columns):
product_fda_list_name          95183 non-null object
state_code                     95185 non-null object
sum_total_amount_reimbursed    95185 non-null float64
rank                           95185 non-null int64
dtypes: float64(1), int64(1), object(2)
memory usage: 6.1+ MB


Create a new dataframe with the top 10 drugs in each state.

In [15]:
top_10_amount = amount[amount["rank"] <= 10]
top_10_amount.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 510 entries, 171 to 94892
Data columns (total 4 columns):
product_fda_list_name          510 non-null object
state_code                     510 non-null object
sum_total_amount_reimbursed    510 non-null float64
rank                           510 non-null int64
dtypes: float64(1), int64(1), object(2)
memory usage: 19.9+ KB


How many times does each drug appear in a state's top-10 list?

In [16]:
counts_amount = top_10_amount["product_fda_list_name"].value_counts().reset_index() # Create new dataframe of drug counts
counts_amount.columns = ["product_fda_list_name", "count"] # Rename columns
counts_amount.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 2 columns):
product_fda_list_name    110 non-null object
count                    110 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.8+ KB


Merge the dataframes into a single dataframe with both ranks and counts.

In [17]:
top_10_amount = top_10_amount.merge(counts_amount, how="inner", on="product_fda_list_name")
top_10_amount.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 510 entries, 0 to 509
Data columns (total 5 columns):
product_fda_list_name          510 non-null object
state_code                     510 non-null object
sum_total_amount_reimbursed    510 non-null float64
rank                           510 non-null int64
count                          510 non-null int64
dtypes: float64(1), int64(2), object(2)
memory usage: 23.9+ KB


Which drugs appear in only a single state's top-10 list?

In [18]:
outliers_amount = top_10_amount[top_10_amount["count"] == 1]
outliers_amount.sort_values("product_fda_list_name", ascending=True)

,product_fda_list_name,state_code,sum_total_amount_reimbursed,rank,count
411,ABILIFY 10,MT,2534711.31,7,1
366,ADVAIR HFA,ME,3750001.53,9,1
212,ADVATE 5ML,NV,12833381.30,3,1
404,ARANESP (D,SD,1961736.07,9,1
501,COMPLERA,NJ,20757871.33,10,1
417,COMPLERA T,DC,5734734.51,9,1
34,DEXAMETHAS,VA,11047921.43,10,1
503,DEXTROAMP-,MA,24321167.39,6,1
418,DULERA INH,AL,12122809.43,6,1
499,EPCLUSA,NH,2185523.57,7,1


Export the outliers data as an Excel file.

In [19]:
outliers_amount.to_excel("data/outliers_amount.xlsx")

## How many doctors on Medicaid drug utilization boards have received payments from drug manufacturers?

First, connect to the database.

In [34]:
with open("config.json") as f:
    conf = json.load(f)

In [35]:
conn_str = "host={} dbname={} user={} password={}".format(conf["host"], conf["database"], conf["user"], conf["password"])

In [36]:
conn = psycopg2.connect(conn_str)

Query the database to return the doctors who received any general purpose payments from drug manufacturers.

In [37]:
general_payments = pd.read_sql("""SELECT dur_committee_members.*,
       open_payments_general.physician_first_name as first,
       open_payments_general.physician_middle_name as middle,
       open_payments_general.physician_last_name as last,
       open_payments_general.recipient_city as city,
       open_payments_general.recipient_state as state,
       open_payments_general.physician_specialty as specialty,
       open_payments_general.Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID as manufacturer_id,
       open_payments_general.Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name as manufacturer_name,
       open_payments_general.Total_Amount_of_Payment_USDollars as amount,
       open_payments_general.Date_of_Payment as payment_date,
       open_payments_general.Form_of_Payment_or_Transfer_of_Value as payment_form,
       open_payments_general.Nature_of_Payment_or_Transfer_of_Value as nature_of_payment,
       open_payments_general.Record_ID as record_id
FROM dur_committee_members
INNER JOIN open_payments_general ON dur_committee_members.first_name = open_payments_general.physician_first_name
AND dur_committee_members.last_name = open_payments_general.physician_last_name
AND dur_committee_members.state = open_payments_general.recipient_state""",
                           con=conn)

In [38]:
general_payments["amount"] = pd.to_numeric(general_payments["amount"], errors="raise") # Convert the amount column to float data type
general_payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371 entries, 0 to 1370
Data columns (total 30 columns):
first_name             1371 non-null object
last_name              1371 non-null object
middle_initial         272 non-null object
degree_1               1308 non-null object
degree_2               64 non-null object
state                  1371 non-null object
dur_pt_other           1371 non-null object
info_date              1371 non-null object
date_minutes_or_web    1371 non-null object
industry_rep           0 non-null object
location               561 non-null object
specialty              101 non-null object
committee_alt_name     442 non-null object
notes                  346 non-null object
disclosure_received    211 non-null object
conflict_disclosed     4 non-null object
conflict_details       4 non-null object
first                  1371 non-null object
middle                 1176 non-null object
last                   1371 non-null object
city                   1371 no

Do we have duplicates in the committee data?

In [39]:
general_payments.groupby("record_id")["record_id"].count().sort_values(ascending=False)

record_id
293514490    2
150145894    2
103835238    2
256381878    2
149553884    2
102393188    2
255343784    2
293514492    2
255846404    2
150271840    1
150290398    1
150311072    1
150320696    1
150330082    1
9882196      1
150332946    1
150334248    1
150263950    1
150334312    1
150348398    1
150361096    1
150369212    1
150372118    1
150372777    1
150265166    1
150242408    1
150263906    1
150071534    1
150030334    1
150041642    1
150043974    1
150044156    1
150062338    1
150064740    1
150071016    1
150071516    1
150071762    1
150398092    1
150073114    1
150160380    1
150169484    1
150181014    1
150192164    1
150192754    1
150210826    1
150211272    1
150373636    1
150439668    1
150398928    1
150414859    1
150594646    1
150609704    1
150624492    1
150653422    1
150665410    1
150668774    1
150683968    1
150687511    1
150690556    1
150694720    1
150700864    1
150712738    1
150714054    1
150724130    1
150726566    1
150730688    1


We do. Let's drop them.

In [40]:
general_payments.drop_duplicates("record_id", inplace=True)
general_payments.groupby("record_id")["record_id"].count().sort_values(ascending=False)

record_id
9882196      1
150330082    1
150242408    1
150263906    1
150263950    1
150265166    1
150271840    1
150290398    1
150311072    1
150320696    1
150332946    1
150398928    1
150334248    1
150334312    1
150348398    1
150361096    1
150369212    1
150372118    1
150372777    1
150373636    1
150211272    1
150210826    1
150192754    1
150192164    1
150024636    1
150025124    1
150030334    1
150041642    1
150043974    1
150044156    1
150062338    1
150064740    1
150071016    1
150071516    1
150071534    1
150071762    1
150073114    1
150145894    1
150160380    1
150169484    1
150181014    1
150398092    1
150414859    1
149196156    1
150700864    1
150624492    1
150653422    1
150665410    1
150668774    1
150683968    1
150687511    1
150690556    1
150694720    1
150712738    1
150433498    1
150714054    1
150724130    1
150726566    1
150730688    1
150736060    1
150763674    1
150775462    1
150789082    1
150609704    1
150594646    1
150593646    1


What is the total value of general purpose payments to doctors serving on Medicaid drug utilization boards?

In [41]:
general_payments["amount"].sum()

143262.4000000001

Query the database to return the doctors who held an ownership or investment interest in any drug manufacturers.

In [46]:
ownership_payments = pd.read_sql("""SELECT dur_committee_members.*,
       open_payments_ownership.physician_first_name as first,
       open_payments_ownership.physician_middle_name as middle,
       open_payments_ownership.physician_last_name as last,
       open_payments_ownership.recipient_city as city,
       open_payments_ownership.recipient_state as state,
       open_payments_ownership.physician_specialty as specialty,
       open_payments_ownership.Total_Amount_Invested_USDollars as total_amount_invested,
       open_payments_ownership.Value_of_Interest as value_of_interest,
       open_payments_ownership.Terms_of_Interest as terms_of_interest,
       open_payments_ownership.Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID as manufacturer_id,
       open_payments_ownership.Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name as manufacturer_name,
       open_payments_ownership.Interest_Held_by_Physician_or_an_Immediate_Family_Member as nature_of_interest,
       open_payments_ownership.Record_ID as record_id
FROM dur_committee_members
INNER JOIN open_payments_ownership ON dur_committee_members.first_name = open_payments_ownership.physician_first_name
AND dur_committee_members.last_name = open_payments_ownership.physician_last_name
AND dur_committee_members.state = open_payments_ownership.recipient_state""",
                           con=conn)

In [48]:
# Convert the investment amount columns to float data type
ownership_payments["total_amount_invested"] = pd.to_numeric(ownership_payments["total_amount_invested"], errors="raise")
ownership_payments["value_of_interest"] = pd.to_numeric(ownership_payments["value_of_interest"], errors="raise")
ownership_payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 30 columns):
first_name               2 non-null object
last_name                2 non-null object
middle_initial           0 non-null object
degree_1                 2 non-null object
degree_2                 0 non-null object
state                    2 non-null object
dur_pt_other             2 non-null object
info_date                2 non-null object
date_minutes_or_web      2 non-null object
industry_rep             0 non-null object
location                 0 non-null object
specialty                1 non-null object
committee_alt_name       0 non-null object
notes                    1 non-null object
disclosure_received      0 non-null object
conflict_disclosed       0 non-null object
conflict_details         0 non-null object
first                    2 non-null object
middle                   1 non-null object
last                     2 non-null object
city                     2 non-null obj

Do we have duplicates in the committee data?

In [49]:
ownership_payments.groupby("record_id")["record_id"].count().sort_values(ascending=False)

record_id
413359278    1
152335554    1
Name: record_id, dtype: int64

In [50]:
ownership_payments

,first_name,last_name,middle_initial,degree_1,degree_2,state,dur_pt_other,info_date,date_minutes_or_web,industry_rep,location,specialty,committee_alt_name,notes,disclosure_received,conflict_disclosed,conflict_details,first,middle,last,city,state,specialty,total_amount_invested,value_of_interest,terms_of_interest,manufacturer_id,manufacturer_name,nature_of_interest,record_id
0,Andreas,Wali,None,MD,None,PA,P&T,3/1/18,From Web,None,None,cardiologist,None,effective Oct 11 2017,None,None,None,Andreas,U,Wali,Camp Hill,PA,Allopathic & Osteopathic Physicians|Internal M...,19086.85,10607.38,Stock ownership,100000000133,TriReme Medical LLC,Physician Covered Recipient,413359278
1,Sharon,Weinstein,None,MD,None,UT,DUR,12/8/16,From Minutes,None,None,None,None,None,None,None,None,Sharon,None,Weinstein,Salt Lake City,UT,Allopathic & Osteopathic Physicians|Psychiatry...,0.00,0.00,None,100000005587,"DARA Biosciences, Inc.",Physician Covered Recipient,152335554


What is the total value of the ownership or investment interest of doctors serving on Medicaid drug utilization boards?

In [51]:
ownership_payments["total_amount_invested"].sum()

19086.85

Query the database to return the doctors who received any research payments from drug manufacturers.

In [ ]:
general_payments = pd.read_sql("""SELECT dur_committee_members.*,
       open_payments_general.physician_first_name as first,
       open_payments_general.physician_middle_name as middle,
       open_payments_general.physician_last_name as last,
       open_payments_general.recipient_city as city,
       open_payments_general.recipient_state as state,
       open_payments_general.physician_specialty as specialty,
       open_payments_general.Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID as manufacturer_id,
       open_payments_general.Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name as manufacturer_name,
       open_payments_general.Total_Amount_of_Payment_USDollars as amount,
       open_payments_general.Date_of_Payment as payment_date,
       open_payments_general.Form_of_Payment_or_Transfer_of_Value as payment_form,
       open_payments_general.Expenditure_Category1 as expenditure_category_1,
       open_payments_general.Expenditure_Category2 as expenditure_category_2,
       open_payments_general.Expenditure_Category3 as expenditure_category_3,
       open_payments_general.Expenditure_Category4 as expenditure_category_4,
       open_payments_general.Expenditure_Category5 as expenditure_category_5,
       open_payments_general.Expenditure_Category6 as expenditure_category_6,
       open_payments_general.Nature_of_Payment_or_Transfer_of_Value as nature_of_payment,
       open_payments_general.Record_ID as record_id
FROM dur_committee_members
INNER JOIN open_payments_general ON dur_committee_members.first_name = open_payments_general.physician_first_name
AND dur_committee_members.last_name = open_payments_general.physician_last_name
AND dur_committee_members.state = open_payments_general.recipient_state""",
                           con=conn)

Export each of the dataframes to an Excel file for further analysis.

In [ ]:
writer = pd.ExcelWriter("data/doc_payments.xlsx")
general_payments.to_excel(writer, "general_payments", index=False)
ownership_payments.to_excel(writer, "ownership_payments", index=False)
research_payments.to_excel(writer, "research_payments", index=False)
writer.save()